## Task 1

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
from data.cf_loader import Task1Loader, Task2Loader

/home/nas4_user/hawonjeong/anaconda3/envs/won/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]= "3"
torch.cuda.is_available()

True

In [3]:
task1_dataset = Task1Loader('/home/nas3_userL/sohyunjeong/work_dir/etc/23spring/hw2/dataset/user_itemset_training.csv').data_preprocess()
valid_df = pd.read_csv('/home/nas3_userL/sohyunjeong/work_dir/etc/23spring/hw2/dataset/user_itemset_valid_query.csv', names=['user_id', 'itemset_id'])
valid_ans = pd.read_csv('/home/nas3_userL/sohyunjeong/work_dir/etc/23spring/hw2/dataset/user_itemset_valid_answer.csv', names=['answer'])
print("Loading data is done.")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 53897/53897 [00:30<00:00, 1782.20it/s]


Loading data is done.


In [4]:
NUM_USERS = 53897
NUM_ITEMSETS = 27694
latent_dim = 4096

In [6]:
user_rep = torch.zeros((NUM_USERS, latent_dim))

for user in tqdm(range(NUM_USERS)):
    checkpoint_path = f'/home/nas4_user/saemeechoi/course/DataMining/WCL/data/dataset/features_task1/{user}.pt'
    checkpoint = torch.load(checkpoint_path, map_location='cpu')
    user_rep[user] = checkpoint

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 53897/53897 [02:41<00:00, 332.79it/s]


In [7]:
mul_user_rep = torch.mm(user_rep, user_rep.t())

norm = torch.sqrt(torch.diag(mul_user_rep)).unsqueeze(1)
user_sim = torch.div(mul_user_rep, norm)
user_sim = torch.div(user_sim, norm.t())
user_sim[torch.eye(user_sim.size(0)).bool()] = 0

In [ ]:
task1_dataset_tensor = torch.tensor(task1_dataset).float()
valid_df['score'] = torch.zeros(len(valid_df))

# if torch.cuda.is_available():
#     user_sim = user_sim.to('cuda')
#     task1_dataset_tensor = task1_dataset_tensor.to('cuda')

for i in tqdm(range(len(valid_df))):
    user_id = valid_df['user_id'][i]
    itemset_id = valid_df['itemset_id'][i]

    numer = torch.matmul(user_sim[user_id].unsqueeze(0), task1_dataset_tensor[:, itemset_id].unsqueeze(1))
    denom = torch.sum(user_sim[user_id])

    valid_df['score'][i] = numer.item() / denom.item()

  0%|                                                                                                                                                                                                     | 0/335940 [00:00<?, ?it/s]/tmp/ipykernel_2418547/3170230859.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df['score'][i] = numer.item() / denom.item()
 28%|████████████████████████████████████████████████████▎                                                                                                                                   | 95590/335940 [04:27<08:21, 479.70it/s]

#### ▼▼▼ Evaluation on validation set

In [ ]:
from sklearn.metrics import accuracy_score

y_score = np.array(valid_df['score'])
y_true = np.array(valid_ans['answer'])

thresholds = np.linspace(0, 0.005, num=100)

accuracy_values = []
max_acc = 0
max_threshold = 0

for threshold in thresholds:
    # Convert scores to binary labels based on threshold
    y_pred = [1 if score >= threshold else 0 for score in y_score]
    
    # Calculate evaluation metrics
    accuracy = accuracy_score(y_true, y_pred)
    
    if accuracy > max_acc:
        max_acc = accuracy
        max_threshold = threshold

    # Append metric values to lists
    accuracy_values.append(accuracy)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(thresholds, accuracy_values, label='Accuracy')
plt.xlabel('Threshold')
plt.ylabel('Accuracy')
plt.title('accuracy vs Threshold')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# task 1
print(max_acc)
print(max_threshold)

#### ▼▼▼ Run code on test set

In [19]:
test_df = pd.read_csv('/home/nas3_userL/sohyunjeong/work_dir/etc/23spring/hw2/dataset/user_itemset_test_query.csv', names=['user_id', 'itemset_id'])
test_df['score'] = torch.zeros(len(test_df))

In [ ]:
if torch.cuda.is_available():
    user_sim = user_sim.to('cuda')
    task1_dataset_tensor = task1_dataset_tensor.to('cuda')

In [21]:
for i in tqdm(range(len(test_df))):
    wcl_id = test_df[test_df.columns[0]][i]
    weight_id = test_df[test_df.columns[1]][i]

    numer = torch.matmul(user_sim[wcl_id].unsqueeze(0), task1_dataset_tensor[:, weight_id].unsqueeze(1))
    denom = torch.sum(user_sim[wcl_id])

    test_df['score'][i] = numer.item() / denom.item()

  0%|                                                                                                                                                                                                     | 0/335940 [00:00<?, ?it/s]/tmp/ipykernel_2361513/415710219.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['score'][i] = numer.item() / denom.item()
 79%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                                      | 266145/335940 [07:40<02:00, 578.35it/s]


KeyboardInterrupt: 

In [ ]:
test_df.to_csv('/home/nas4_user/hawonjeong/courses/ai506/DM_termproj/results/user_itemset_test_prediction.csv', index=False)

## Task 2

In [24]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
from data.cf_loader import Task1Loader, Task2Loader

In [25]:
os.environ["CUDA_VISIBLE_DEVICES"]= "3"
torch.cuda.is_available()

True

In [88]:
task2_dataset = Task2Loader('/home/nas3_userL/sohyunjeong/work_dir/etc/23spring/hw2/dataset/itemset_item_training.csv').data_preprocess()
task2_valid_dataset = Task2Loader('/home/nas3_userL/sohyunjeong/work_dir/etc/23spring/hw2/dataset/itemset_item_valid_query.csv').data_preprocess()
valid_df = pd.read_csv('/home/nas3_userL/sohyunjeong/work_dir/etc/23spring/hw2/dataset/itemset_item_valid_query.csv', names=['itemset_id', 'item_id'])
valid_ans = pd.read_csv('/home/nas3_userL/sohyunjeong/work_dir/etc/23spring/hw2/dataset/itemset_item_valid_answer.csv', names=['itemset_id', 'item_id'])

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2769/2769 [00:00<00:00, 5521.03it/s]


In [90]:
NUM_USERS = 53897
NUM_ITEMSETS = 27694
latent_dim = 4096
NUM_ITEMS = 42653

In [91]:
wcl_rep = torch.zeros((NUM_ITEMSETS, latent_dim))

for itemset in tqdm(range(NUM_ITEMSETS)):
    checkpoint_path = f'/home/nas4_user/saemeechoi/course/DataMining/WCL/data/dataset/features_task2_2/{itemset}.pt'
    checkpoint = torch.load(checkpoint_path, map_location='cpu')
    wcl_rep[itemset] = checkpoint
print("Checkpoints are loaded.")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27694/27694 [01:09<00:00, 399.47it/s]

Checkpoints are loaded.


In [93]:
# Get similarity matrix
mul_wcl_rep = torch.mm(wcl_rep, wcl_rep.t())
norm = torch.sqrt(torch.diag(mul_wcl_rep)).unsqueeze(1)
sim = torch.div(mul_wcl_rep, norm)
sim = torch.div(sim, norm.t())
sim[torch.eye(sim.size(0)).bool()] = 0

In [95]:
# >>>>> To get multi-hot representation of all items
# 1. Training set
df = pd.read_csv('/home/nas3_userL/sohyunjeong/work_dir/etc/23spring/hw2/dataset/itemset_item_training.csv', names=['itemset_id', 'item_id'])
itemsets = list(set(df['itemset_id'].tolist()))

data_dict = dict()
for itemset in tqdm(itemsets):
    items = df[df['itemset_id']==itemset]['item_id'].tolist()
    data_dict.update({itemset:items})

itemset_item = np.zeros((NUM_ITEMSETS, NUM_ITEMS))    
for i, (k, v) in enumerate(data_dict.items()) :
    itemset_item[k, v] = 1

# 2. Valid set
df = pd.read_csv('/home/nas3_userL/sohyunjeong/work_dir/etc/23spring/hw2/dataset/itemset_item_valid_query.csv', names=['itemset_id', 'item_id'])
itemsets = list(set(df['itemset_id'].tolist()))

data_dict = dict()
for itemset in tqdm(itemsets):
    items = df[df['itemset_id']==itemset]['item_id'].tolist()
    data_dict.update({itemset:items})

for i, (k, v) in enumerate(data_dict.items()) :
    itemset_item[k,v] = 1

# 3. Test set
df = pd.read_csv('/home/nas3_userL/sohyunjeong/work_dir/etc/23spring/hw2/dataset/itemset_item_test_query.csv', names=['itemset_id', 'item_id'])
itemsets = list(set(df['itemset_id'].tolist()))

data_dict = dict()
for itemset in tqdm(itemsets):
    items = df[df['itemset_id']==itemset]['item_id'].tolist()
    data_dict.update({itemset:items})

for i, (k, v) in enumerate(data_dict.items()) :
    itemset_item[k,v] = 1

# <<<<< End of getting multi-hot representation of all items
print(itemset_item.shape)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2769/2769 [00:00<00:00, 5605.59it/s]


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
itemset_item_tensor = torch.tensor(itemset_item).float()

if torch.cuda.is_available():
    sim = sim.to('cuda')
    itemset_item_tensor = itemset_item_tensor.to('cuda')

In [99]:
# valid_ans['candidate_items'][i] = [item1_score, item2_score, ..., item100_score]
valid_ans['candidate_items'] = None

for i in tqdm(range(len(valid_ans))):
    itemset_id = valid_ans[valid_ans.columns[0]][i]
    
    # get scores of all items
    numer = torch.matmul(sim[itemset_id].unsqueeze(0), itemset_item_tensor)
    denom = torch.sum(sim[itemset_id])
    item_scores = numer.squeeze() / denom.item()
    
    top_100_items = torch.argsort(item_scores, descending=True)[:100]
    valid_ans['candidate_items'][i] = top_100_items.cpu().numpy()

  0%|                                                                                                                                                                                                       | 0/2769 [00:00<?, ?it/s]/tmp/ipykernel_2195381/2099983369.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_ans['candidate_items'][i] = top_100_items.cpu().numpy()
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2769/2769 [00:44<00:00, 62.08it/s]


In [100]:
valid_ans

,itemset_id,item_id,candidate_items
0,6998,14100,"[31432, 34338, 25194, 13979, 19204, 27447, 737..."
1,8941,37713,"[31432, 34338, 25194, 19204, 13979, 27447, 129..."
2,11717,29703,"[31432, 27447, 13979, 25194, 34338, 19204, 364..."
3,18595,37718,"[31432, 34338, 25194, 13979, 19204, 27447, 129..."
4,23064,42559,"[34338, 31432, 25194, 19204, 13979, 16926, 346..."
...,...,...,...
2764,396,11105,"[25194, 19204, 13979, 16926, 27447, 31432, 364..."
2765,7525,35558,"[34338, 31432, 25194, 19204, 13979, 4191, 2744..."
2766,14719,24249,"[31432, 34338, 25194, 13979, 19204, 27447, 169..."
2767,23768,10764,"[31432, 34338, 25194, 13979, 19204, 27447, 737..."


#### ▼▼▼ Evaluation on validation set

In [ ]:
# task 2 accuracy
from sklearn.metrics import accuracy_score

y_pred = [0 for i in range(len(valid_ans))]
y_pred_rank = [101 for i in range(len(valid_ans))]

for idx in range(len(valid_ans['itemset_id'])):
    
    item_id = valid_ans['item_id'][idx]
    candidate_items = valid_ans['candidate_items'][idx]
    
    if np.isin(item_id, candidate_items):
        y_pred_rank[idx] = np.where(candidate_items == item_id)[0].item()
        y_pred[idx] = 1
        
# accuracy
# y_true = list(valid_ans['item_id'])
accuracy = sum(y_pred) / len(valid_ans)
average_rank = np.mean(y_pred_rank)

In [ ]:
print("Accuracy: ", accuracy)
print("Average rank: ", average_rank)

#### ▼▼▼ Run code on test set

In [ ]:
test_df = pd.read_csv('/home/nas3_userL/sohyunjeong/work_dir/etc/23spring/hw2/dataset/itemset_item_test_query.csv', names=['itemset_id', 'item_id'])

In [ ]:
# itemset_item: caculated at validation
itemset_item_tensor = torch.tensor(itemset_item).float()

if torch.cuda.is_available():
    sim = sim.to('cuda')
    itemset_item_tensor = itemset_item_tensor.to('cuda')

In [ ]:
test_df['candidate_items'] = None

for i in tqdm(range(len(test_df))):
    itemset_id = test_df[test_df.columns[0]][i]

    # get scores of all items
    numer = torch.matmul(sim[itemset_id].unsqueeze(0), itemset_item_tensor)
    denom = torch.sum(sim[itemset_id])
    item_scores = numer.squeeze() / denom.item()

    top_100_items = torch.argsort(item_scores, descending=True)[:100]
    test_df['candidate_items'][i] = top_100_items.cpu().numpy()

In [ ]:
# For submission
test_sub = test_df.copy()
test_sub.drop("item_id", axis = 1)

print(test_sub)
test_sub.to_csv('/home/nas4_user/hawonjeong/courses/ai506/DM_termproj/results/itemset_item_test_prediction.csv', index=False)